## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-02-13-46 +0000,startribune,Phoenix eliminates shorthanded Lynx from WNBA ...,https://www.startribune.com/phoenix-eliminates...
1,2025-09-29-02-06-00 +0000,nyt,Afghanistan Frees a U.S. Citizen Held in Prison,https://www.nytimes.com/2025/09/28/world/asia/...
2,2025-09-29-02-02-44 +0000,nyt,"Trump Raises Possibility of Inquiry Into Wray,...",https://www.nytimes.com/2025/09/28/us/politics...
3,2025-09-29-02-02-06 +0000,nyt,"With Scant Information, Federal Workers Brace ...",https://www.nytimes.com/2025/09/28/us/politics...
4,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2293/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,22
52,shooting,13
51,church,13
29,new,11
50,michigan,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,74
10,2025-09-29-01-25-43 +0000,bbc,At least four dead and several injured in shoo...,https://www.bbc.com/news/articles/ceq2vd15glwo...,65
24,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...,60
6,2025-09-29-01-51-06 +0000,wapo,"At least 4 dead, 8 injured in Michigan church ...",https://www.washingtonpost.com/nation/2025/09/...,60
4,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,57


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,74,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
6,60,2025-09-29-01-51-06 +0000,wapo,"At least 4 dead, 8 injured in Michigan church ...",https://www.washingtonpost.com/nation/2025/09/...
24,60,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
31,32,2025-09-28-23-26-36 +0000,nypost,Mississippi man shows up drunk to bail out pal...,https://nypost.com/2025/09/28/us-news/mississi...
84,28,2025-09-28-19-48-52 +0000,nypost,Iraq War veteran Thomas Sanford ID’d as gunman...,https://nypost.com/2025/09/28/us-news/iraq-war...
192,26,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
50,25,2025-09-28-21-46-12 +0000,nypost,30K runners retrace 9/11 hero’s path in NYC Tu...,https://nypost.com/2025/09/28/us-news/30k-runn...
48,23,2025-09-28-21-59-11 +0000,startribune,Three pedestrians struck and killed in Twin Ci...,https://www.startribune.com/man-killed-after-b...
158,22,2025-09-28-11-00-47 +0000,wapo,Court dismisses Central American bank’s claim ...,https://www.washingtonpost.com/world/2025/09/2...
172,21,2025-09-28-09-08-40 +0000,nypost,"Russia fires over 600 drones, missiles in dead...",https://nypost.com/2025/09/28/world-news/large...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
